<a href="https://colab.research.google.com/github/voidnire/DeepLearning/blob/cursos/SarcasmsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers --quiet
!pip install opendatasets --quiet

import opendatasets as od
od.download("https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: erindante
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection


100%|██████████| 3.30M/3.30M [00:00<00:00, 706MB/s]

In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Available Device is ", device)

Available Device is  cuda


In [4]:
data_df = pd.read_json("/content/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json", lines=True)
data_df.dropna(inplace = True)
data_df.drop_duplicates(inplace = True)

print(data_df.shape)
data_df.head()

(26708, 3)


,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [5]:
SEED = 23

In [31]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data_df['headline']),
                                                    np.array(data_df['is_sarcastic']),
                                                    test_size=0.2)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

print("Training Size: ", X_train.shape[0], "- ",round(X_train.shape[0]/data_df.shape[0],4)*100,"%")
print("Validation Size: ", X_val.shape[0], "- ",round(X_val.shape[0]/data_df.shape[0],4)*100,"%")
print("Testing Size: ", X_test.shape[0], "- ",round(X_test.shape[0]/data_df.shape[0],4)*100,"%")


Training Size:  21366 -  80.0 %
Validation Size:  2671 -  10.0 %
Testing Size:  2671 -  10.0 %


In [32]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
bert_model = AutoModel.from_pretrained('google-bert/bert-base-uncased')

In [33]:
class dataset(Dataset):
  def __init__(self, X, Y):
    self.X = [tokenizer(x,
                        max_length = 100,
                        truncation = True,
                        padding = "max_length", # [1,3,2,0,0,0,0,0,..,0]
                        return_tensors = "pt" #pytorch tensors
                        ).to(device) for x in X
    ]
    self.Y = torch.tensor(Y, dtype = torch.float32).to(device)
    #self.tokenizer = tokenizer

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


In [34]:
training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

- Video:
https://www.youtube.com/watch?v=E0bwEAWmVEM

HYPERPARAMS

In [35]:
BATCH_SIZE = 32
EPOCHS = 3
LR = 1e-4

In [36]:
train_dataloader = DataLoader(training_data, batch_size = BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(validation_data, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(testing_data, batch_size = BATCH_SIZE, shuffle = True)

In [37]:
class MyModel(nn.Module):
  def __init__(self,bert):
    super(MyModel,self).__init__()
    self.bert = bert_model
    self.dropout = nn.Dropout(0.25)

    self.linear1 = nn.Linear(768,384)
    self.linear2 = nn.Linear(384,1)
    #self.relu = nn.ReLU()


  def forward(self, input_ids, attention_mask):
    outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict=True
    )

    pooled_output = outputs.last_hidden_state[:, 0, :] # CLS
    output = self.linear1(pooled_output)
    output = self.dropout(output)
    output = self.linear2(output)
    return output

In [38]:
for param in bert_model.parameters():
  param.requires_grad = False

model = MyModel(bert_model).to(device)


In [39]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3
)



#Adam(model.parameters(), lr = LR)

In [40]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []

for epoch in range(EPOCHS):
  total_loss_train = 0
  total_acc_train = 0
  total_loss_validation = 0
  total_acc_validation = 0

  for indx, data in enumerate(train_dataloader):
    inputs, labels = data
    #inputs.to(device)
    #labels.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    labels = labels.to(device)


    prediction = model(inputs["input_ids"].squeeze(1), inputs["attention_mask"].squeeze(1)).squeeze(1)
    batch_loss = criterion(prediction, labels)
    total_loss_train += batch_loss.item()

    acc = (prediction.round() == labels).sum().item()
    total_acc_train += acc

    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()


  with torch.no_grad():
    for indx, data in enumerate(val_dataloader):
      inputs, labels = data
      inputs = {k: v.to(device) for k, v in inputs.items()}
      labels = labels.to(device)

      prediction = model(inputs["input_ids"].squeeze(1), inputs["attention_mask"].squeeze(1)).squeeze(1)
      batch_loss = criterion(prediction, labels)
      total_loss_validation += batch_loss.item()

      acc = (prediction.round() == labels).sum().item()
      total_acc_validation += acc


### Teste

In [ ]:
total_loss_train_plot.append(round(total_loss_train/1000,4))
total_loss_validation_plot.append(round(total_loss_validation/1000,4))

total_acc_train_plot.append((round(total_acc_train/training_data.__len__()) *100,4))
total_acc_validation_plot.append((round(total_acc_validation/validation_data.__len__()) *100,4))

In [ ]:
print(f"""
Epoch No. {epoch+1}
Training Loss: {round(total_loss_train/1000,4)}
Train Accuracy: {round((total_acc_train/training_data.__len__())*100,4)}


Validation Loss: {round(total_loss_validation/1000,4)}
Validation Accuracy: {round((total_acc_validation/validation_data.__len__())*100,4)}
""")

In [ ]:
with torch.no_grad():
  total_loss_test = 0
  total_acc_test = 0

  for indx, data in enumerate(testing_data):
      inputs, labels = data
      inputs = {k: v.to(device) for k, v in inputs.items()}
      labels = labels.to(device)

      prediction = model(inputs["input_ids"].squeeze(1), inputs["attention_mask"].squeeze(1)).squeeze(1)
      batch_loss = criterion(prediction, labels)
      total_loss_test += batch_loss.item()

      acc = (prediction.round() == labels).sum().item()
      total_acc_test += acc

print(f"""

Accuracy on Testing Data: {round((total_acc_test/testing_data.__len__())*100,4)}""")


### Plottando

In [ ]:
fig, axs  = plt.subplots(nrows =1,ncols = 2, figsize=(12,8))

axs[0].plot(total_loss_train_plot, label = "Training Loss")
axs[0].plot(total_loss_validation_plot, label = "Validation Loss")
axs[0].set_title('Training and Val Loss over Epochs')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].set_ylim([0,1])
axs[0].legend()

axs[1].plot(total_acc_train_plot, label = "Training Accuracy")
axs[1].plot(total_acc_validation_plot, label = "Validation Accuracy")
axs[1].set_title('Training and Val Loss over Epochs')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].set_ylim([0,100])
axs[1].legend()

plt.tight_layout()
plt.show()
